In [88]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
!pip install torchinfo

### Preprocess Climate Data

In [90]:
import pandas as pd

file_path = '/content/drive/MyDrive/NNProject/climate_data.csv'

climate_df = pd.read_csv(file_path)
climate_df['date'] = pd.to_datetime(climate_df['date'])
climate_df.index = climate_df['date']

In [91]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = climate_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('lat')
numeric_cols.remove('lon')
non_numeric_cols = climate_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['lat', 'lon']

climate_df[numeric_cols] = climate_df[numeric_cols].astype(float)
climate_df[non_numeric_cols] = climate_df[non_numeric_cols]
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(climate_df[numeric_cols]), columns=numeric_cols, index=climate_df.index)

climate_data = pd.concat([scaled_df, climate_df[non_numeric_cols]], axis=1).reset_index(drop= True)


In [92]:

print(climate_data["lat"].unique())
print(climate_data["lon"].unique())

[10. 18. 14. 22.]
[72. 80. 76. 84.]


### Preprocess Energy Data

In [93]:
file_path = '/content/drive/MyDrive/NNProject/long_data_.csv'

energy_df = pd.read_csv(file_path)
energy_df['Dates'] = pd.to_datetime(energy_df['Dates'], format='%d/%m/%Y %H:%M:%S')
energy_df.index = energy_df['Dates']

redundant_cols = ["Regions"]
energy_df = energy_df.drop(columns=redundant_cols)


In [94]:

numeric_cols = energy_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('latitude')
numeric_cols.remove('longitude')
non_numeric_cols = energy_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['latitude', 'longitude']
energy_df[numeric_cols] = energy_df[numeric_cols].astype(float)

scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(energy_df[numeric_cols]), columns=numeric_cols, index=energy_df.index)
energy_data = pd.concat([scaled_df, energy_df[non_numeric_cols]], axis=1).reset_index(drop=True)

### State-Lat-Lon Map

In [ ]:

import numpy as np

lat_range = np.arange(10, 36, 4)
lon_range = np.arange(72, 92, 4)

energy_data["latitude"] = energy_data["latitude"].apply(lambda x: lat_range[np.argmin(np.abs(lat_range - x))])
energy_data["longitude"] = energy_data["longitude"].apply(lambda x: lon_range[np.argmin(np.abs(lon_range - x))])

## PreTrain on Climate Data

### Time-Series Sequences

In [96]:
import numpy as np

def lagged_climate_input(data, seq_length=28):
    X = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_values("date")
        features = group.drop(columns=["date", "lat", "lon"])
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
    return np.array(X)

climate_X = lagged_climate_input(climate_data, 14)


### Model

In [97]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ClimatePreTrainer(nn.Module):
    def __init__(self, input_size, hidden_size=64, latent_size=32, num_layers=1, dropout=0.3):
        super(ClimatePreTrainer, self).__init__()

        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                               batch_first=True, bidirectional=True, dropout=dropout)

        self.to_latent = nn.Linear(hidden_size * 2, latent_size)
        self.from_latent = nn.Linear(latent_size, hidden_size * 2)

        self.decoder = nn.LSTM(hidden_size * 2, hidden_size, num_layers=num_layers,
                               batch_first=True, bidirectional=True, dropout=dropout)

        self.output_layer = nn.Linear(hidden_size * 2, input_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        encoded_x, _ = self.encoder(x)
        encoded_x = self.dropout(encoded_x)

        latent = self.to_latent(encoded_x)
        expanded = self.from_latent(latent)

        decoded_x, _ = self.decoder(expanded)
        decoded_x = self.dropout(decoded_x)

        reconstructed_x = self.output_layer(decoded_x)
        return reconstructed_x, latent


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchinfo import summary

num_epochs = 50
batch_size = 64
lr = 0.01

patience = 5
best_val_loss = float('inf')
counter = 0

climate_X_tensor = torch.tensor(climate_X, dtype=torch.float32)

train_X, val_X = train_test_split(climate_X_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(train_X)
val_dataset = TensorDataset(val_X)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
climate_pretrained_model = ClimatePreTrainer(input_size=climate_X.shape[2], num_layers=2).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(climate_pretrained_model.parameters(), lr=lr, weight_decay=1e-5)

for epoch in range(num_epochs):
    climate_pretrained_model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch[0].to(device)
        optimizer.zero_grad()

        reconstructed_x, _ = climate_pretrained_model(batch)
        loss = criterion(reconstructed_x, batch)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    climate_pretrained_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch[0].to(device)
            reconstructed_x, _ = climate_pretrained_model(batch)
            loss = criterion(reconstructed_x, batch)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(climate_pretrained_model.state_dict(), 'climate_pretrained_model.pth')
        print(f"Validation loss improved. Saving model.")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience Counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break


Epoch [1/50], Train Loss: 0.004214, Val Loss: 0.000861
Validation loss improved. Saving model.
Epoch [2/50], Train Loss: 0.001405, Val Loss: 0.000699
Validation loss improved. Saving model.
Epoch [3/50], Train Loss: 0.001437, Val Loss: 0.001471
⚠️ Validation loss did not improve. Patience Counter: 1/5
Epoch [4/50], Train Loss: 0.001482, Val Loss: 0.000790
⚠️ Validation loss did not improve. Patience Counter: 2/5
Epoch [5/50], Train Loss: 0.001454, Val Loss: 0.000925
⚠️ Validation loss did not improve. Patience Counter: 3/5
Epoch [6/50], Train Loss: 0.001453, Val Loss: 0.000784
⚠️ Validation loss did not improve. Patience Counter: 4/5
Epoch [7/50], Train Loss: 0.002292, Val Loss: 0.001192
⚠️ Validation loss did not improve. Patience Counter: 5/5
Early stopping triggered.


#### Freeze Encoder Model

In [98]:
climate_pretrained_model = ClimatePreTrainer(input_size=climate_X.shape[2], num_layers=2).to(device)
climate_pretrained_model.load_state_dict(torch.load('climate_pretrained_model.pth'))

for param in climate_pretrained_model.parameters():
    param.requires_grad = False

## FineTune Downstream

### Merge DataSources

In [99]:
energy_data = energy_data.rename(columns={"latitude": "lat", "longitude": "lon"})

merged_df = pd.merge(
    energy_data,
    climate_data,
    how="left",
    left_on=["lat", "lon", "Dates"],
    right_on=["lat", "lon", "date"],
    suffixes=("", "_climate"),
)

In [100]:
merged_df.shape

(16599, 17)

In [101]:
merged_df = merged_df.dropna()
merged_df.shape

(5533, 17)

In [102]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = merged_df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col])


In [103]:

def lagged_merged_data(data, seq_length=28):
    X = []
    y = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_values("date")
        features = group.drop(columns=["date", "Dates", "lat", "lon", "States", "Usage"])
        target = group["Usage"]
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
            y.append(target.iloc[row+seq_length])
    return np.array(X), np.array(y)

merged_X, targt_y = lagged_merged_data(merged_df, 14)

In [104]:

class EnergyPrediction(nn.Module):
    def __init__(self, encoder, input_size, hidden_size=64):
        super(EnergyPrediction, self).__init__()
        self.encoder = encoder.encoder
        self.regressor = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        encoded_x, _ = self.encoder(x)  # [batch, seq_len, hidden*2]
        x = self.regressor(encoded_x[:, -1, :])  # Use last timestep
        return x  # [batch, 1]


In [105]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(merged_X, targt_y, test_size=0.2, random_state=42)


In [106]:
from torch.utils.data import DataLoader, TensorDataset
from torchinfo import summary

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

climate_pretrained_model.eval()
energy_model = EnergyPrediction(climate_pretrained_model, input_size=merged_X.shape[2]).to(device)
print("Model Summary:")
print(energy_model)
print(summary(energy_model, input_size=(1, 14, merged_X.shape[2])))

Model Summary:
EnergyPrediction(
  (encoder): LSTM(11, 64, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (regressor): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)
Layer (type:depth-idx)                   Output Shape              Param #
EnergyPrediction                         [1, 1]                    --
├─LSTM: 1-1                              [1, 14, 128]              (138,752)
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 64]                   8,256
│    └─ReLU: 2-2                         [1, 64]                   --
│    └─Linear: 2-3                       [1, 1]                    65
Total params: 147,073
Trainable params: 8,321
Non-trainable params: 138,752
Total mult-adds (Units.MEGABYTES): 1.95
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.59
Est

In [107]:
lr = 0.001
num_epochs = 50

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(energy_model.parameters(), lr=lr)
energy_model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        targets = targets.view(-1, 1)
        optimizer.zero_grad()
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/50], Loss: 0.0690
Epoch [2/50], Loss: 0.0241
Epoch [3/50], Loss: 0.0779
Epoch [4/50], Loss: 0.0300
Epoch [5/50], Loss: 0.0552
Epoch [6/50], Loss: 0.0504
Epoch [7/50], Loss: 0.0686
Epoch [8/50], Loss: 0.0587
Epoch [9/50], Loss: 0.0656
Epoch [10/50], Loss: 0.0791
Epoch [11/50], Loss: 0.0949
Epoch [12/50], Loss: 0.0170
Epoch [13/50], Loss: 0.0253
Epoch [14/50], Loss: 0.0706
Epoch [15/50], Loss: 0.0302
Epoch [16/50], Loss: 0.0508
Epoch [17/50], Loss: 0.0387
Epoch [18/50], Loss: 0.0300
Epoch [19/50], Loss: 0.0560
Epoch [20/50], Loss: 0.0685
Epoch [21/50], Loss: 0.0401
Epoch [22/50], Loss: 0.0192
Epoch [23/50], Loss: 0.0237
Epoch [24/50], Loss: 0.0438
Epoch [25/50], Loss: 0.1081
Epoch [26/50], Loss: 0.0737
Epoch [27/50], Loss: 0.1195
Epoch [28/50], Loss: 0.0219
Epoch [29/50], Loss: 0.0415
Epoch [30/50], Loss: 0.1194
Epoch [31/50], Loss: 0.0119
Epoch [32/50], Loss: 0.0372
Epoch [33/50], Loss: 0.0591
Epoch [34/50], Loss: 0.0194
Epoch [35/50], Loss: 0.0632
Epoch [36/50], Loss: 0.0405
E

In [108]:

for param in energy_model.encoder.parameters():
    param.requires_grad = True

print("Model Summary:")
print(energy_model)
print(summary(energy_model, input_size=(1, 14, merged_X.shape[2])))

for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        targets = targets.view(-1, 1)
        optimizer.zero_grad()
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Model Summary:
EnergyPrediction(
  (encoder): LSTM(11, 64, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (regressor): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)
Layer (type:depth-idx)                   Output Shape              Param #
EnergyPrediction                         [1, 1]                    --
├─LSTM: 1-1                              [1, 14, 128]              138,752
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 64]                   8,256
│    └─ReLU: 2-2                         [1, 64]                   --
│    └─Linear: 2-3                       [1, 1]                    65
Total params: 147,073
Trainable params: 147,073
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.95
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.59
Estimated

In [109]:
energy_model.eval()
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
val_loss = 0
with torch.no_grad():
    for batch in val_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        targets = targets.view(-1, 1)
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        val_loss += loss.item()
avg_val_loss = val_loss / len(val_loader)
print(f"Validation Loss: {avg_val_loss:.4f}")

Validation Loss: 0.0356
